In [44]:
#Kraabime veebilehelt PDFid ja slaidiesitlused maha

import os
import requests
from urllib.parse import urljoin
from time import sleep 
from selenium import webdriver
from bs4 import BeautifulSoup
#Tegin siia meetodi, mis võtab nasdaqbalticu lehelt siis ISIN koodi ja kuupäeva järgi kõik .pdf formaadis failid,
#laeb need lokaalselt mulle alla.
def fetch_pdfs():
    #If there is no such folder, the script will create one automatically
    folder_location = r'C:\Users\marek.keskull\Documents\webscraping\Luminor'
    if not os.path.exists(folder_location):os.mkdir(folder_location)
    # Set some Selenium Options
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    # Webdriver
    #wd = webdriver.Chrome('chromedriver',options=options)
    wd = webdriver.Chrome(executable_path=r'C:\Users\marek.keskull\Documents\GitHub\NLP\chromedriver_win32\chromedriver.exe')
    # URL
    url = f"https://luminor.ee/investorsuhted"
    # Load URL
    wd.get(url)
    sleep(7)
    # Get HTML
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    # Try to find all hidden data
    hidden = soup.find('div', {'class': 'content-widget-wrapper'})

    for link in hidden.select("a[href$='.pdf']"):
        filename = os.path.join(folder_location,link['href'].split('/')[-1])
        print(filename)
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link['href'])).content)
fetch_pdfs()

C:\Users\marek.keskull\Documents\webscraping\Luminor\q4-2020-eng-17.02.2021.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_bank_as_interim_report_q3_2020.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_bank_as_interim_report_q2_2020.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_bank_as_interim_report_q1_2020.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_q4_2019_interim_report_en.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_q3_2019_interim_report_en.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_q2_2019_interim_report_eng.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_bank_as_estonia_q1_2019_en.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-ee-q4-eng.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-lv-q4-eng.pdf
C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-lt-q4-eng.pdf
C:\Users\marek.keskull\Documents\webscrap

In [45]:
#kasutan tika libraryt, selle peab panema lokaalselt käima eraldi, et kasutada tika rest teenust.
import tika
from tika import parser

import os

folder_location = r'C:\Users\marek.keskull\Documents\webscraping\Luminor'
#Võtan kõik allalaetud PDF'id ning muudan need .txt failideks, et mul jääks alles ainult plain text
def extract_text_from_pdfs_recursively(dir):
    for root, dirs, files in os.walk(dir):
        for file in files:
            path_to_pdf = os.path.join(root, file)
            [stem, ext] = os.path.splitext(path_to_pdf)
            if ext == '.pdf':
                print("Processing " + path_to_pdf)
                pdf_contents = parser.from_file(path_to_pdf)
                path_to_txt = stem + '.txt'
                with open(path_to_txt, 'w',encoding="utf-8") as txt_file:
                    print("Writing contents to " + path_to_txt)
                    txt_file.write(pdf_contents['content'])
                    


if __name__ == "__main__":
    extract_text_from_pdfs_recursively(folder_location)

Processing C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-ee-2q-report-2018-en.pdf
Writing contents to C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-ee-2q-report-2018-en.txt
Processing C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-ee-3q-report-2018-en.pdf
Writing contents to C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-ee-3q-report-2018-en.txt
Processing C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-ee-4q-report-2018-en.pdf
Writing contents to C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor-ee-4q-report-2018-en.txt
Processing C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_bank_as_interim_report_q1_2020.pdf
Writing contents to C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_bank_as_interim_report_q1_2020.txt
Processing C:\Users\marek.keskull\Documents\webscraping\Luminor\luminor_bank_as_interim_report_q2_2020.pdf
Writing contents to C:\Users\marek.keskull\Documents\webscrap

In [46]:
quarters = []
basedir = r"C:\Users\marek.keskull\Documents\webscraping"
path = os.path.join(basedir, folder_location)
files = os.listdir(path)
for file in files:
    path_to_pdf = os.path.join(path, file)
    [stem, ext] = os.path.splitext(path_to_pdf)
    if ext == '.txt':
        quarters.append(file)

In [47]:
quarters

['luminor-ee-2q-report-2018-en.txt',
 'luminor-ee-3q-report-2018-en.txt',
 'luminor-ee-4q-report-2018-en.txt',
 'luminor_bank_as_interim_report_q1_2020.txt',
 'luminor_bank_as_interim_report_q2_2020.txt',
 'luminor_bank_as_interim_report_q3_2020.txt',
 'luminor_bank_as_interim_report_q4_2020.txt',
 'luminor_q1_2019_interim_report_en.txt',
 'luminor_q2_2019_interim_report_eng.txt',
 'luminor_q3_2019_interim_report_en.txt',
 'luminor_q4_2019_interim_report_en.txt']

In [48]:
data = []
for c in quarters:
    with open(path+ "\\" + c, "rb") as file:
        text = file.read()
        asd = text.strip()
        decoded=str(asd,'utf-8')
        data.append(decoded)
data

["LUMINOR GROUP AB \r\nINTERIM CONSOLIDATED ADMINISTRATION REPORT, \r\nINTERIM CONDENSED FINANCIAL INFORMATION \r\nFOR THE PERIOD ENDED 30 JUNE 2018 \r\n \r\n\r\n \r\n\r\n\r\n\r\n \r\n\r\nPage 2 of 35 \r\n\r\n \r\n\r\nCONTENTS \r\n \r\n\r\nPage \r\n\r\nLUMINOR GROUP AB CONSOLIDATED ADMINISTRATION REPORT FOR THE HALF YEAR 2018 3 \r\n\r\nCONDENSED CONSOLIDATED INCOME STATEMENT 7 \r\n\r\nCONDENSED CONSOLIDATED STATEMENT OF COMPREHENSIVE INCOME 8 \r\n\r\nCONDENSED CONSOLIDATED BALANCE SHEET 9 \r\n\r\nCONDENSED CONSOLIDATED CHANGES IN SHAREHOLDERS' EQUITY 10 \r\n\r\nCONDENSED CONSOLIDATED CASH FLOW STATEMENT 11 \r\n\r\nCONDENSED PARENT COMPANY'S INCOME STATEMENT 12 \r\n\r\nCONDENSED PARENT COMPANY'S BALANCE SHEET 13 \r\n\r\nCONDENSED PARENT COMPANY'S CHANGES IN SHAREHOLDERS' EQUITY 14 \r\n\r\nCONDENSED PARENT COMPANY'S CASH FLOW STATEMENT 15 \r\n\r\nNOTES TO FINANCIAL STATEMENTS 16 \r\n\r\nACCOUNTING PRINCIPLES 16 \r\n\r\nOTHER NOTES TO THE FINANCIAL STATEMENTS 24 \r\n \r\n\r\n \r\n \r\n\r\

In [49]:
#teen listidest dictionary
ddata = dict(zip(quarters, data))
ddata

{'luminor-ee-2q-report-2018-en.txt': "LUMINOR GROUP AB \r\nINTERIM CONSOLIDATED ADMINISTRATION REPORT, \r\nINTERIM CONDENSED FINANCIAL INFORMATION \r\nFOR THE PERIOD ENDED 30 JUNE 2018 \r\n \r\n\r\n \r\n\r\n\r\n\r\n \r\n\r\nPage 2 of 35 \r\n\r\n \r\n\r\nCONTENTS \r\n \r\n\r\nPage \r\n\r\nLUMINOR GROUP AB CONSOLIDATED ADMINISTRATION REPORT FOR THE HALF YEAR 2018 3 \r\n\r\nCONDENSED CONSOLIDATED INCOME STATEMENT 7 \r\n\r\nCONDENSED CONSOLIDATED STATEMENT OF COMPREHENSIVE INCOME 8 \r\n\r\nCONDENSED CONSOLIDATED BALANCE SHEET 9 \r\n\r\nCONDENSED CONSOLIDATED CHANGES IN SHAREHOLDERS' EQUITY 10 \r\n\r\nCONDENSED CONSOLIDATED CASH FLOW STATEMENT 11 \r\n\r\nCONDENSED PARENT COMPANY'S INCOME STATEMENT 12 \r\n\r\nCONDENSED PARENT COMPANY'S BALANCE SHEET 13 \r\n\r\nCONDENSED PARENT COMPANY'S CHANGES IN SHAREHOLDERS' EQUITY 14 \r\n\r\nCONDENSED PARENT COMPANY'S CASH FLOW STATEMENT 15 \r\n\r\nNOTES TO FINANCIAL STATEMENTS 16 \r\n\r\nACCOUNTING PRINCIPLES 16 \r\n\r\nOTHER NOTES TO THE FINANCIAL STAT

In [50]:
def combine_text(list_of_text):
    combined_text = ''.join(list_of_text)
    return combined_text

In [51]:
data_combined = {key: [combine_text(value)] for (key, value) in ddata.items()}

In [52]:
#pandas dataframe data
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['text']
data_df = data_df.sort_index()
data_df

,text
luminor-ee-2q-report-2018-en.txt,"LUMINOR GROUP AB \r\nINTERIM CONSOLIDATED ADMINISTRATION REPORT, \r\nINTERIM CONDENSED FINANCIAL INFORMATION \r\nFOR THE PERIOD ENDED 30 JUNE 2018..."
luminor-ee-3q-report-2018-en.txt,1\r\n\r\nInterim Report \r\nfor Q3 2018\r\n \r\n\r\nThe interim report has been prepared in \r\naccordance with the IAS 34 and requirements set \r...
luminor-ee-4q-report-2018-en.txt,Luminor EE Q4 eng_naudit\r\n\r\n\r\n \r\n\r\n1 \r\n \r\n\r\n1 \r\n\r\n \r\n\r\nINTERIM REPORT FOR \r\nQ4 2018 \r\nThe interim report has been prep...
luminor_bank_as_interim_report_q1_2020.txt,1 \r\n\r\nl \r\n\r\n \r\n\r\nINTERIM REPORT \r\n\r\nQ1 2020 \r\nThe interim report has been prepared in \r\n\r\naccordance with the IAS 34 Inter...
luminor_bank_as_interim_report_q2_2020.txt,1 \r\n\r\nmmovidal \r\n\r\n \r\n\r\nINTERIM REPORT \r\n\r\nQ2 2020 \r\nThe interim report has been prepared in \r\naccordance with the IAS 34 In...
luminor_bank_as_interim_report_q3_2020.txt,Microsoft Word - Q3 2020 ENG_Final\r\n\r\n\r\n \r\n\r\n \r\n\r\n1 \r\n\r\nmmovidal \r\n\r\n \r\n\r\nINTERIM REPORT \r\n\r\nQ3 2020 \r\nThe inter...
luminor_bank_as_interim_report_q4_2020.txt,Microsoft Word - Q4 2020 ENG 03.02.2021\r\n\r\n\r\n \r\n\r\n \r\n\r\n1 \r\n\r\nmmovidal \r\n\r\n \r\n\r\nINTERIM REPORT \r\n\r\nQ4 2020 \r\nThe ...
luminor_q1_2019_interim_report_en.txt,1 \r\n\r\n \r\n\r\nINTERIM REPORT \r\n\r\nFOR Q1 2019 \r\nThe interim report has been prepared in \r\n\r\naccordance with the IAS 34 and \r\n\r\n...
luminor_q2_2019_interim_report_eng.txt,1 \r\n\r\nINTERIM REPORT \r\n\r\nFOR Q2 2019 \r\nThe interim report has been prepared in \r\n\r\naccordance with the IAS 34 and \r\n\r\nrequiremen...
luminor_q3_2019_interim_report_en.txt,1 \r\n\r\n \r\n\r\nINTERIM REPORT \r\n\r\nQ3 2019 \r\nThe interim report has been prepared in \r\n\r\naccordance with the IAS 34 Interim \r\n\r\n...


In [53]:
import re
import string

def clean_text_round1(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

round1 = lambda x: clean_text_round1(x)

In [54]:
#vaatame puhastatud teksti
data_clean2 = pd.DataFrame(data_df.text.apply(round1))
data_clean2

,text
luminor-ee-2q-report-2018-en.txt,luminor group ab \r\ninterim consolidated administration report \r\ninterim condensed financial information \r\nfor the period ended june \r\n ...
luminor-ee-3q-report-2018-en.txt,\r\n\r\ninterim report \r\nfor \r\n \r\n\r\nthe interim report has been prepared in \r\naccordance with the ias and requirements set \r\nby the ...
luminor-ee-4q-report-2018-en.txt,luminor ee eng naudit\r\n\r\n\r\n \r\n\r\n \r\n \r\n\r\n \r\n\r\n \r\n\r\ninterim report for \r\n \r\nthe interim report has been prepared in ac...
luminor_bank_as_interim_report_q1_2020.txt,\r\n\r\nl \r\n\r\n \r\n\r\ninterim report \r\n\r\n \r\nthe interim report has been prepared in \r\n\r\naccordance with the ias interim \r\n\r...
luminor_bank_as_interim_report_q2_2020.txt,\r\n\r\nmmovidal \r\n\r\n \r\n\r\ninterim report \r\n\r\n \r\nthe interim report has been prepared in \r\naccordance with the ias interim \r\...
luminor_bank_as_interim_report_q3_2020.txt,microsoft word eng final\r\n\r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\nmmovidal \r\n\r\n \r\n\r\ninterim report \r\n\r\n \r\nthe interim report has...
luminor_bank_as_interim_report_q4_2020.txt,microsoft word eng \r\n\r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\nmmovidal \r\n\r\n \r\n\r\ninterim report \r\n\r\n \r\nthe interim report has be...
luminor_q1_2019_interim_report_en.txt,\r\n\r\n \r\n\r\ninterim report \r\n\r\nfor \r\nthe interim report has been prepared in \r\n\r\naccordance with the ias and \r\n\r\nrequireme...
luminor_q2_2019_interim_report_eng.txt,\r\n\r\ninterim report \r\n\r\nfor \r\nthe interim report has been prepared in \r\n\r\naccordance with the ias and \r\n\r\nrequirements set by...
luminor_q3_2019_interim_report_en.txt,\r\n\r\n \r\n\r\ninterim report \r\n\r\n \r\nthe interim report has been prepared in \r\n\r\naccordance with the ias interim \r\n\r\nfinancial...


In [55]:
# 2. round puhastamisele
def clean_text_round2(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    #text = re.sub('https?://\S+|www\.\S+', '', text)
    #text = re.sub('<.*?>+', '', text)
    text = re.sub('\r', ' ', text)
    #text = re.sub(r"\s+"," ", text)
    #text = re.sub(r"^\s+", "", text)
    #text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)
    #text = re.sub(r'/\d\.\s+|[a-z]\)\s+|•\s+|[A-Z]\.\s+|[IVX]+\.\s+/g', "",text)
    #text = re.sub(r'\xc2\xb7',"",text)
    text = re.sub(r'\W+', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [56]:
#vaatame puhastatud teksti
data_clean2 = pd.DataFrame(data_clean2.text.apply(round2))
pd.options.display.max_colwidth = 500
data_clean2

,text
luminor-ee-2q-report-2018-en.txt,luminor group ab interim consolidated administration report interim condensed financial information for the period ended june page of contents page luminor group ab consolidated administration report for the half year condensed consolidated income statement condensed consolidated statement of comprehensive income condensed consolidated balance sheet condensed consolidated changes in shareholders equity condensed consolidated cash flow statement condensed parent company s income statement con...
luminor-ee-3q-report-2018-en.txt,interim report for the interim report has been prepared in accordance with the ias and requirements set by the bank of estonia for quarterly reporting by credit institutions luminor bank as estonia table of contents management report condensed consolidated interim financial statements condensed consolidated interim statement of profit or loss and other comprehensive income condensed consolidated statement of financial position as at september condensed consolidated statement of changes in e...
luminor-ee-4q-report-2018-en.txt,luminor ee eng naudit interim report for the interim report has been prepared in accordance with the ias and requirements set by the bank of estonia for quarterly reporting by credit institutions luminor bank as estonia table of contents management report condensed consolidated interim financial statements condensed consolidated statement of profit or loss and other comprehensive income condensed consolidated statement of financial position as at december condensed consolidated statement of ...
luminor_bank_as_interim_report_q1_2020.txt,l interim report the interim report has been prepared in accordance with the ias interim financial reporting and requirements set by the bank of estonia for quarterly reporting by credit institutions luminor bank as estonia luminor bank as interim report for the period ended march contents ceo comment management report condensed consolidated interim financial statements notes to the condensed consolidated interim financial statements contact details luminor bank as interim report for the pe...
luminor_bank_as_interim_report_q2_2020.txt,mmovidal interim report the interim report has been prepared in accordance with the ias interim financial reporting and requirements set by the bank of estonia for quarterly reporting by credit institutions luminor bank as estonia luminor bank as interim report for the period ended june contents ceo comment management report condensed consolidated interim financial statements notes to the condensed consolidated interim financial statements contact details luminor bank as interim report for ...
luminor_bank_as_interim_report_q3_2020.txt,microsoft word eng final mmovidal interim report the interim report has been prepared in accordance with the ias interim financial reporting and requirements set by the bank of estonia for quarterly reporting by credit institutions luminor bank as estonia luminor bank as interim report for the period ended september contents ceo comment management report condensed consolidated interim financial statements notes to the condensed consolidated interim financial statements contact details lumino...
luminor_bank_as_interim_report_q4_2020.txt,microsoft word eng mmovidal interim report the interim report has been prepared in accordance with the ias interim financial reporting and requirements set by the bank of estonia for quarterly reporting by credit institutions luminor bank as estonia luminor bank as interim report for the period ended december contents ceo comment management report condensed consolidated interim financial statements condensed consolidated statement of profit or loss and other comprehensive income condensed co...
luminor_q1_2019_interim_report_en.txt,interim report for the interim report has been prepared in accordance with the ias and requirements set by the bank of estonia for quarterly reporting by credit institutions luminor bank a

In [57]:
import pickle
data_clean2.to_pickle("pickles/Luminor2018t2020.pkl")